In [5]:
#import libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input ,GlobalMaxPool1D
from keras.layers import Conv1D, MaxPool1D ,Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score



In [6]:
# some configuration
max_sequence_length=100
max_vocab_size=20000
embedding_dim=100
validation_split=.2
batch_size=128
epochs=10


In [9]:
## load  pre trained vector
word2vec={}
with open('glove/glove.6B.%sd.txt' % embedding_dim,encoding="utf8") as f:
    for line in f:
        values=line.split()
        word=values[0]
        vec=np.asarray(values[1:],dtype='float32')
        word2vec[word]=vec

In [14]:
##load data
train=pd.read_csv('toxic-comment-classification-challenge/train.csv')
#train.head()
sentences=train['comment_text'].fillna("DUMMY_VALUE").values
possible_labels=["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
targest=train[possible_labels].values


In [16]:
#covert sentences to integer
tokenizer=Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(sentences)
sequences=tokenizer.texts_to_sequences(sentences)
#sequences[0]


In [17]:
## get word2int 
word2idx= tokenizer.word_index

In [19]:
## pad sequences
data=pad_sequences(sequences,maxlen=max_sequence_length)


In [23]:
###prepare embedding metrix
num_words=min(max_vocab_size,len(word2idx)+1)
embedding_matrix=np.zeros((num_words,embedding_dim))
for word,i in word2idx.items():
    if i< max_vocab_size:
        embedding_vector=word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector

In [24]:
## embedding layer
embedding_layer=Embedding(num_words,embedding_dim,weights=[embedding_matrix],input_length=max_sequence_length,trainable=False)


In [26]:
##building model 
input_=Input(shape=(max_sequence_length,))
x=embedding_layer(input_)
x=Conv1D(128,3,activation='relu')(x)
x=MaxPool1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=MaxPool1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPool1D()(x)
x=Dense(128,activation='relu')(x)
output=Dense(len(possible_labels),activation='sigmoid')(x)
model=Model(input_,output)
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(data,targest,batch_size=batch_size,epochs=epochs,validation_split=validation_split)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 253s 2ms/step - loss: 0.0830 - acc: 0.9735 - val_loss: 0.0735 - val_acc: 0.9763
Epoch 2/10
127656/127656 [==============================] - 241s 2ms/step - loss: 0.0672 - acc: 0.9773 - val_loss: 0.0692 - val_acc: 0.9768
Epoch 3/10
127656/127656 [==============================] - 235s 2ms/step - loss: 0.0629 - acc: 0.9784 - val_loss: 0.0691 - val_acc: 0.9762
Epoch 4/10
127656/127656 [==============================] - 233s 2ms/step - loss: 0.0598 - acc: 0.9792 - val_loss: 0.0708 - val_acc: 0.9780
Epoch 5/10
127656/127656 [==============================] - 228s 2ms/step - loss: 0.0575 - acc: 0.9797 - val_loss: 0.0781 - val_acc: 0.9778
Epoch 6/10
127656/127656 [==============================] - 230s 2ms/step - loss: 0.0552 - acc: 0.9807 - val_loss: 0.0738 - val_acc: 0.9776
Epoch 7/10
127656/127656 [==============================] - 229s 2ms/step - loss: 0.0540 - acc: 0.9810 - val_